In [6]:
import pandas as pd
import openai
import os
import gspread
import numpy as np
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAIKEY')

In [9]:
def read_url(url, SHEET_NAME):
    SHEET_ID = url.split("/")[5]
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    rows = worksheet.get_all_records()
    df_spread = pd.DataFrame(rows)
    return df_spread, worksheet

In [10]:
serviceaccount = "../../google_drive_personal.json"
gc = gspread.service_account(filename=serviceaccount)
GC_URL = "https://docs.google.com/spreadsheets/d/1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw/edit?usp=sharing"

In [11]:
SHEETNAME = "select_city_classifier"
city_meta, other_worksheet = read_url(GC_URL, SHEETNAME)
city_meta = city_meta[city_meta['City']!=''].reset_index(drop = True)
city_meta["country_clean"] = np.where(city_meta["Country"].isin(["USA", "United States"]), 
                                      "United States of America",
                                      city_meta["Country"]
                                     )

In [7]:

def get_completion(messages, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    # messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [22]:
city_meta_str = city_meta.apply(lambda x: f"{x['City']}, {x['country_clean']}", axis=1).tolist()

input_city = ";".join(city_meta_str[:2])
input_city

'Hindupur, India;Vijayawada, India'

In [42]:
prompt = """
Your task is to:
Read a city and country combinations below inside the triple backquote. \
The city and country are separated by a comma. \
Input examples is: "city_1, country_1".
Given the input city and country combination, you should return the state or province of the city and country combination in the format like:\
"state: returned state or province".\
Input city and country combinations:```{input_city}```
"""
# messages = [{"role": "user", "content": prompt}]
# answer = get_completion(messages)
# # # messages.append({"role": "assistant", "content": answer})
# answer

In [47]:
from tqdm import tqdm
allstates = []
for input_city in tqdm(city_meta_str):
    # prompt.format(input_city = input_city)
    messages = [{"role": "user", "content": prompt.format(input_city = input_city)}]
    answer = get_completion(messages)
    print(answer)
    allstates.append(answer)

  1%|          | 1/127 [00:00<01:42,  1.22it/s]

Output: state: Andhra Pradesh


  2%|▏         | 2/127 [00:01<01:43,  1.21it/s]

Output: state: Andhra Pradesh


  2%|▏         | 3/127 [00:02<01:27,  1.41it/s]

...

Output:
state: Jakarta


 99%|█████████▉| 126/127 [01:18<00:00,  2.02it/s]

Output: state: Bangkok


100%|██████████| 127/127 [01:18<00:00,  1.61it/s]

state: Illinois


In [49]:
allstates_clean = [x.split(":")[-1].strip() for x in allstates]
allstates_clean

['Andhra Pradesh',
 'Andhra Pradesh',
 'Rangpur Division',
 'British Columbia',
 'Santiago',
 'Cesar',
 'Perm Krai',
 'Rivne',
 'West Java',
 'West Java',
 'Tel Aviv District',
 'Bahia',
 'Metro Manila',
 'Gombe',
 'Guatemala',
 'Maharashtra',
 'Pays de la Loire',
 'Cochabamba',
 'Bahia',
 'Nizhny Novgorod Oblast',
 'Uttar Pradesh',
 'Astrakhan Oblast',
 'Overijssel',
 'Tyumen Oblast',
 'Texas',
 'California',
 'Negros Occidental',
 'Sicily',
 'Cebu',
 'Florida',
 'Central Region',
 'Florida',
 'Kaunas County',
 'California',
 'Greater Manchester',
 'Île-de-France',
 'Jalisco',
 'Tamaulipas',
 'Belgrade',
 'Minnesota',
 'Antwerp',
 'Greater Accra',
 'Tocantins',
 'Sinaloa',
 'Ohio',
 'Maharashtra',
 'Maharashtra',
 'Rajshahi',
 'Kerala',
 'Ohio',
 'Santa Catarina',
 'Wellington',
 'Massachusetts',
 'England',
 'Sao Paulo',
 'North Sumatra',
 'Bavaria',
 'South-East',
 'Jerusalem District',
 'South Sumatra',
 'Maharashtra',
 'Madrid',
 'Stockholm County',
 'Kyiv City',
 'Taipei City',
 

In [51]:
city_meta["State/Province"] = allstates_clean

In [52]:
other_worksheet.update([city_meta.columns.values.tolist()] + city_meta.values.tolist())

{'spreadsheetId': '1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw',
 'updatedRange': 'select_city_classifier!A1:V128',
 'updatedRows': 128,
 'updatedColumns': 22,
 'updatedCells': 2816}